In [1]:
import re
import spacy
import neuralcoref
from pycorenlp import *
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

/home/ky/anaconda3/envs/thxldll/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/home/ky/anaconda3/envs/thxldll/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/home/ky/anaconda3/envs/thxldll/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [2]:
standford_nlp = StanfordCoreNLP("http://localhost:9000/")
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [3]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    if text.strip().endswith("<stop>"):
        sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [4]:
def neuralcorefIt(text):
    sentences = split_into_sentences(text)
#     sentences[0] = sentences[0].capitalize()
    for s in sentences:
        if s[-1] == '?':
            sentences.remove(s)
    result = ""
    for sentence in sentences:
        doc = nlp(sentence)
        result += " " + doc._.coref_resolved
    return result.strip()

In [5]:
def getPresentTense(triples):
    lmtzr = WordNetLemmatizer()
    for t in range(0, len(triples)):
        for i in range(0,len(triples[t])):
            for word in triples[t][i].split(" "):
                triples[t][i] = triples[t][i].replace(word, lmtzr.lemmatize(word,'v'))
    return triples

In [6]:
def produceTriples(text):
    processedText = neuralcorefIt(text)
    output = standford_nlp.annotate(processedText,
                          properties={"annotators":"tokenize,ssplit,pos,lemma,depparse,natlog,openie,dcoref",
                                      "outputFormat": "json","triple.strict":"true"})
    triples = []
    result2 = []
    result = [output["sentences"][0]["openie"] for item in output]
    for i in range(0,len(output["sentences"])):
        result2.append(output['sentences'][i]['openie'])
        for i in range(0, len(result2)):
            for rel in result2[i]:
                subj = rel['subject']
                obj = rel['object']
                for ref in output['corefs']:
                    if len(output['corefs'][ref]) > 1 and output['corefs'][ref][1]['position'][0] == i + 1:
                        if output['corefs'][ref][1]['text'] == subj:
                            subj = output['corefs'][ref][0]['text']
                relationSent=[subj,obj,rel['relation']]
                triples.append(relationSent)
    return getPresentTense(triples)

In [7]:
def write_to_file(triples):
    out_put_file = "data/triples.txt"
    file_out = open(out_put_file, 'w')
    check = []
    for triple in triples:
        line = "\t".join(triple)
        if line not in check:
            check.append(line)
            file_out.write(line.lower() + "\n")
    file_out.close()

In [8]:
# write_to_file(triples)

In [9]:
def addTriples():
    sentence = "Obama was dead. I am so sad for him."
    triples = produceTriples(sentence)
    print(triples)
    write_to_file(triples)

In [ ]:
addTriples()